<a href="https://colab.research.google.com/github/pratik-poudel/datacrunch/blob/main/last_sub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install optuna

In [6]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import gc
# pd.options.display.float_format| = "{:.2f}".format
pd.options.display.max_columns = 500


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report , mean_squared_error, make_scorer
import requests
from scipy import stats
from numpy import absolute
from numpy import mean
from numpy import std

import lightgbm as lgb

from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDRegressor, ElasticNet

In [7]:
train_datalink_X = 'https://tournament.datacrunch.com/data/X_train.csv'  
train_datalink_y = 'https://tournament.datacrunch.com/data/y_train.csv'
hackathon_data_link = 'https://tournament.datacrunch.com/data/X_test.csv'
train = pd.read_csv(train_datalink_X)
target = pd.read_csv(train_datalink_y)
test = pd.read_csv(hackathon_data_link)

In [8]:
features = [f for f in train.columns if 'feature' in f]

In [9]:
def additional_features(df):
    df['feature_17+feature_23+feature_8*feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_8'] * df['feature_21']
    df['feature_23+feature_4+feature_8+feature_21'] = df['feature_23'] + df['feature_4'] + df['feature_8'] + df['feature_21']
    df['feature_18+feature_23+feature_8*feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_8'] * df['feature_21']
    df['feature_23+feature_11+feature_8+feature_21'] = df['feature_23'] + df['feature_11'] + df['feature_8'] + df['feature_21']
    df['feature_17+feature_18+feature_23+feature_21'] = df['feature_17'] + df['feature_18'] + df['feature_23'] + df['feature_21']
    df['feature_23+feature_12+feature_8+feature_21'] = df['feature_23'] + df['feature_12'] + df['feature_8'] + df['feature_21']
    df['feature_17+feature_19+feature_23+feature_21'] = df['feature_17'] + df['feature_19'] + df['feature_23'] + df['feature_21']
    df['feature_23+feature_5+feature_8+feature_21'] = df['feature_23'] + df['feature_5'] + df['feature_8'] + df['feature_21']
    df['feature_23+feature_3+feature_8+feature_21'] = df['feature_23'] + df['feature_3'] + df['feature_8'] + df['feature_21']
    df['feature_18+feature_19+feature_23+feature_21'] = df['feature_18'] + df['feature_19'] + df['feature_23'] + df['feature_21']
    df['feature_23+feature_9+feature_8+feature_21'] = df['feature_23'] + df['feature_9'] + df['feature_8'] + df['feature_21']
    df['feature_19+feature_23+feature_8*feature_21'] = df['feature_19'] + df['feature_23'] + df['feature_8'] * df['feature_21']
    df['feature_17+feature_23+feature_16*feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_16'] * df['feature_21']
    df['feature_17+feature_23+feature_8+feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_8'] + df['feature_21']
    df['feature_18+feature_23+feature_16*feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_16'] * df['feature_21']
    df['feature_17+feature_23+feature_3*feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_3'] * df['feature_21']
    df['feature_17+feature_23+feature_3+feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_3'] + df['feature_21']
    df['feature_18+feature_23+feature_8+feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_8'] + df['feature_21']
    df['feature_18+feature_23+feature_3*feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_3'] * df['feature_21']
    df['feature_18+feature_23+feature_3+feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_3'] + df['feature_21']
    df['feature_23+feature_8+feature_6+feature_21'] = df['feature_23'] + df['feature_8'] + df['feature_6'] + df['feature_21']
    df['feature_17+feature_23+feature_16+feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_16'] + df['feature_21']
    df['feature_23+feature_3+feature_8*feature_21'] = df['feature_23'] + df['feature_3'] + df['feature_8'] * df['feature_21']
    df['feature_23+feature_8+feature_21+feature_13'] = df['feature_23'] + df['feature_8'] + df['feature_21'] + df['feature_13']
    df['feature_17+feature_23+feature_11*feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_11'] * df['feature_21']
    df['feature_23+feature_8+feature_6*feature_21'] = df['feature_23'] + df['feature_8'] + df['feature_6'] * df['feature_21']
    df['feature_18+feature_23+feature_16+feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_16'] + df['feature_21']
    df['feature_19+feature_23+feature_8+feature_21'] = df['feature_19'] + df['feature_23'] + df['feature_8'] + df['feature_21']
    df['feature_23+feature_8+feature_21*feature_13'] = df['feature_23'] + df['feature_8'] + df['feature_21'] * df['feature_13']
    df['feature_17+feature_8+feature_6+feature_21'] = df['feature_17'] + df['feature_8'] + df['feature_6'] + df['feature_21']
    df['feature_17+feature_23+feature_11+feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_11'] + df['feature_21']
    df['feature_18+feature_23+feature_11*feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_11'] * df['feature_21']
    df['feature_23+feature_8+feature_2*feature_21'] = df['feature_23'] + df['feature_8'] + df['feature_2'] * df['feature_21']
    df['feature_23+feature_8+feature_2+feature_21'] = df['feature_23'] + df['feature_8'] + df['feature_2'] + df['feature_21']
    df['feature_18+feature_8+feature_6+feature_21'] = df['feature_18'] + df['feature_8'] + df['feature_6'] + df['feature_21']
    df['feature_18+feature_23+feature_11+feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_11'] + df['feature_21']
    df['feature_17+feature_23+feature_4*feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_4'] * df['feature_21']
    df['feature_23+feature_11+feature_8*feature_21'] = df['feature_23'] + df['feature_11'] + df['feature_8'] * df['feature_21']
    df['feature_17+feature_5+feature_8+feature_21'] = df['feature_17'] + df['feature_5'] + df['feature_8'] + df['feature_21']
    df['feature_23+feature_22+feature_8+feature_21'] = df['feature_23'] + df['feature_22'] + df['feature_8'] + df['feature_21']
    df['feature_17+feature_23+feature_4+feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_4'] + df['feature_21']
    df['feature_18+feature_23+feature_4*feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_4'] * df['feature_21']
    df['feature_19+feature_23+feature_3+feature_21'] = df['feature_19'] + df['feature_23'] + df['feature_3'] + df['feature_21']
    df['feature_17+feature_19+feature_23*feature_21'] = df['feature_17'] + df['feature_19'] + df['feature_23'] * df['feature_21']
    df['feature_18+feature_5+feature_8+feature_21'] = df['feature_18'] + df['feature_5'] + df['feature_8'] + df['feature_21']
    df['feature_19+feature_23+feature_16*feature_21'] = df['feature_19'] + df['feature_23'] + df['feature_16'] * df['feature_21']
    df['feature_18+feature_19+feature_23*feature_21'] = df['feature_18'] + df['feature_19'] + df['feature_23'] * df['feature_21']
    df['feature_17+feature_23+feature_2*feature_21'] = df['feature_17'] + df['feature_23'] + df['feature_2'] * df['feature_21']
    df['feature_23+feature_4+feature_8*feature_21'] = df['feature_23'] + df['feature_4'] + df['feature_8'] * df['feature_21']
    df['feature_18+feature_23+feature_4+feature_21'] = df['feature_18'] + df['feature_23'] + df['feature_4'] + df['feature_21']
    df['feature_23+feature_8+feature_10+feature_21'] = df['feature_23'] + df['feature_8'] + df['feature_10'] + df['feature_21']
    df['feature_19+feature_23+feature_16+feature_21'] = df['feature_19'] + df['feature_23'] + df['feature_16'] + df['feature_21']
    df['feature_19+feature_8+feature_6+feature_21'] = df['feature_19'] + df['feature_8'] + df['feature_6'] + df['feature_21']
    df['feature_17+feature_18+feature_23*feature_21'] = df['feature_17'] + df['feature_18'] + df['feature_23'] * df['feature_21']
    df['feature_23+feature_8+feature_10*feature_21'] = df['feature_23'] + df['feature_8'] + df['feature_10'] * df['feature_21']
    return df

In [10]:
unique_feature_set = {
    'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_16',
 'feature_17',
 'feature_18',
 'feature_19',
 'feature_2',
 'feature_21',
 'feature_22',
 'feature_23',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_8',
 'feature_9'}

In [11]:
train = additional_features(train)

In [185]:
test = additional_features(test)

In [12]:
features = [f for f in train.columns if 'feature' in f]

In [13]:
def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred))[0]
    # print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))
    return score

In [12]:
params = {'alpha': 0.0364014482430476,
 'fit_intercept': True,
 'l1_ratio': 0.30198788820905625,
 'max_iter': 7085,
 'normalize': False,
 'random_state': 4928,
 'selection': 'random',
 'tol': 0.00814644265439289}
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet
X_train, X_test, y_train, y_test = train_test_split(train[features], target, test_size=0.25, shuffle=False)
model = MultiTaskElasticNet(**params)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print(scorer(y_test['target_r'], preds[:, 0]))
print(scorer(y_test['target_g'], preds[:, 1]))
print(scorer(y_test['target_b'], preds[:, 2]))
print("MEAN SCORE :", (scorer(y_test['target_r'], preds[:, 0]) 
+ scorer(y_test['target_g'], preds[:, 1]) 
+ scorer(y_test['target_b'], preds[:, 2])) / 3)

0.3658408997272413
0.5328872176456995
0.6759614608152857
MEAN SCORE : 0.5248965260627422


In [57]:
import optuna
N_TRIALS = 300
X_train, X_test, y_train, y_test = train_test_split(train[features], target, test_size=0.25, shuffle=False)

def objective(trial):    
    params = {
            'alpha': trial.suggest_uniform('alpha', 0.001, 1),
            'l1_ratio': trial.suggest_uniform('l1_ratio', 0.1, 1),
            'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
            'normalize': trial.suggest_categorical('normalize', [True, False]),
            'max_iter': trial.suggest_int('max_iter', 1000, 10000),
            'tol': trial.suggest_loguniform('tol',1e-7, 1e-2),
            'random_state': trial.suggest_int('random_state', 60, 5000),
            'selection': trial.suggest_categorical('selection', ['cyclic', 'random']),

            }

    model = ElasticNet(**params)
    wrapper = RegressorChain(model, [1, 2, 0])
    wrapper.fit(X_train, y_train)
    preds = wrapper.predict(X_test)
    score = (scorer(y_test['target_r'], preds[:, 0]) +
             scorer(y_test['target_g'], preds[:, 1]) +
             scorer(y_test['target_b'], preds[:, 2])) /3
    from google.colab import output
    output.clear()
    return score
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS)

display(study.best_params)
display(study.best_trial.value)

[W 2021-03-14 05:48:29,260] Trial 299 failed, because the objective function returned nan.


{'alpha': 0.04572179067561144,
 'fit_intercept': True,
 'l1_ratio': 0.3913114457420369,
 'max_iter': 7735,
 'normalize': False,
 'random_state': 4412,
 'selection': 'cyclic',
 'tol': 2.1786679215766246e-05}

0.5219300977933191

In [92]:
X_train, X_test, y_train, y_test = train_test_split(train[features], target, test_size=0.25, shuffle=False)
params = {
'alpha': 0.04572179067561144,
 'fit_intercept': True,
 'l1_ratio': 0.3913114457420369,
 'max_iter': 7735,
 'normalize': False,
 'random_state': 4412,
 'selection': 'cyclic',
 'tol': 2.1786679215766246e-05
 
 }
model = ElasticNet(**params)
wrapper = RegressorChain(model, [1, 2, 0])
wrapper.fit(X_train, y_train)
preds = wrapper.predict(X_test)
print((scorer(y_test['target_r'], preds[:, 0]) + scorer(y_test['target_g'], preds[:, 1]) + scorer(y_test['target_b'], preds[:, 2]))/ 3)

0.5219300977933191


In [74]:
el_preds = pd.DataFrame(wrapper.predict(train[features]), columns=['el_r', 'el_g', 'el_b'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train[features_r], target.target_r, test_size=0.25, shuffle=False, random_state=2021)

def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred))[0]
    print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))
    return score

N_TRIALS = 500
def objective(trial):    
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'num_leaves': trial.suggest_int('num_leaves', 4, 32),
            'max_depth': trial.suggest_int('max_depth', 4, 16),
            'learning_rate' : trial.suggest_uniform('learning_rate', 0.0001, 0.99),
            'n_estimators': trial.suggest_int('n_estimators', 100, 5000),
            'min_split_gain' : trial.suggest_uniform('min_split_gain', 0, 1),
            'min_child_weight': trial.suggest_uniform('min_child_weight', 1e-3, 1),
            'min_child_samples': trial.suggest_int('min_child_samples',20, 500),
            'subsample' : trial.suggest_uniform('subsample', 0.5, 1),
            'subsample_freq': trial.suggest_int('subsample_freq', 0, 1),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),

            }

    model = lgb.LGBMRegressor(**params, random_state=2021)
    # wrapper = RegressorChain(model, [0, 1, 2], random_state=2021)
    # wrapper.fit(X_train, y_train)
    # preds = wrapper.predict(X_test)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=100)
    preds = model.predict(X_test, model.best_iteration_)
    
    
    score = scorer(y_test, preds)
    
    from google.colab import output
    output.clear()
    return score
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar= True)
display(study.best_params)
display(study.best_trial.value)

In [142]:
features_r = ['feature_17+feature_23+feature_8*feature_21',
 'feature_18+feature_23+feature_8*feature_21',
 'feature_17+feature_23+feature_8+feature_21',
 'feature_29',
 'feature_19+feature_23+feature_8*feature_21',
 'feature_18+feature_23+feature_8+feature_21',
 'feature_10',
 'feature_6']

features_g = ['feature_23+feature_11+feature_8+feature_21',
 'feature_23+feature_4+feature_8+feature_21',
 'feature_17+feature_23+feature_8*feature_21',
 'feature_23+feature_3+feature_8+feature_21',
 'feature_17+feature_23+feature_16*feature_21',
 'feature_28',
 'feature_29',
 'feature_25',
 'feature_18+feature_23+feature_8*feature_21',
 'feature_18+feature_23+feature_16+feature_21',
 'feature_26',
 'feature_17+feature_23+feature_3*feature_21',
 'feature_17+feature_18+feature_23*feature_21',
 'feature_23+feature_8+feature_10*feature_21',
 'feature_4',
 'feature_10',
 'feature_19+feature_23+feature_16*feature_21',
 'feature_5',
 'feature_8']

features_b = ['feature_17+feature_23+feature_11*feature_21',
 'feature_17+feature_23+feature_4*feature_21',
 'feature_18+feature_23+feature_4*feature_21',
 'feature_18+feature_23+feature_16*feature_21',
 'feature_18+feature_23+feature_3*feature_21',
 'feature_28',
 'feature_29',
 'feature_25',
 'feature_17',
 'feature_23+feature_22+feature_8+feature_21',
 'feature_10',
 'feature_27']



In [210]:
params_r = {'learning_rate': 0.1881762948600033,
 'max_depth': 13,
 'min_child_samples': 206,
 'min_child_weight': 0.12317444550273453,
 'min_split_gain': 0.6855476015190156,
 'n_estimators': 3207,
 'num_leaves': 28,
 'reg_alpha': 0.000277215555022957,
 'reg_lambda': 0.2112440706134894,
 'subsample': 0.6575876839860837,
 'subsample_freq': 1}

 # 0.3743431445071381

params_g = {'num_leaves': 32, 
 'max_depth': 15,
 'learning_rate': 0.0004900820824294886, 
 'n_estimators': 4982,
 'min_split_gain': 0.9784852734402016, 
 'min_child_weight': 0.3654650150961478,
 'min_child_samples': 453, 
 'subsample': 0.5435998642515718, 
 'subsample_freq': 1, 
 'reg_alpha': 0.0001358772967592931, 
 'reg_lambda': 1.3342132380913252e-05}
# 0.541219083180964
 
params_b = {'learning_rate': 0.09934904889912213,
 'max_depth': 15,
 'min_child_samples': 91,
 'min_child_weight': 0.7401219517079427,
 'min_split_gain': 0.46238343889524225,
 'n_estimators': 1465,
 'num_leaves': 5,
 'reg_alpha': 0.007739393570563738,
 'reg_lambda': 0.0003214724916951722,
 'subsample': 0.8324856340437796,
 'subsample_freq': 0}
#0.6814260043800434

In [211]:
X_train, X_test, y_train, y_test = train_test_split(train[features_r], target.target_r, test_size=0.25, shuffle=False, random_state=2021)

model_target_r = lgb.LGBMRegressor(**params_r, random_state=2021)

model_target_r.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=100)

preds_r = model_target_r.predict(X_test, model_target_r.best_iteration_)
scorer(y_test, preds_r)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.111665
Early stopping, best iteration is:
[17]	valid_0's l2: 0.111664
Score as calculated for the leader board (っಠ‿ಠ)っ 0.3743431445071381


0.3743431445071381

In [215]:
X_train, X_test, y_train, y_test = train_test_split(train[features_g], target.target_g, test_size=0.25, shuffle=False, random_state=2021)

model_target_g = lgb.LGBMRegressor(**params_g, random_state=2021)

model_target_g.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=100)

preds_g = model_target_g.predict(X_test, model_target_g.best_iteration_)

scorer(y_test, preds_g)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.121243
[200]	valid_0's l2: 0.118455
[300]	valid_0's l2: 0.115908
[400]	valid_0's l2: 0.113583
[500]	valid_0's l2: 0.111461
[600]	valid_0's l2: 0.109521
[700]	valid_0's l2: 0.107748
[800]	valid_0's l2: 0.106123
[900]	valid_0's l2: 0.10464
[1000]	valid_0's l2: 0.103284
[1100]	valid_0's l2: 0.102043
[1200]	valid_0's l2: 0.100911
[1300]	valid_0's l2: 0.0998731
[1400]	valid_0's l2: 0.0989262
[1500]	valid_0's l2: 0.0980575
[1600]	valid_0's l2: 0.0972633
[1700]	valid_0's l2: 0.0965356
[1800]	valid_0's l2: 0.0958676
[1900]	valid_0's l2: 0.0952581
[2000]	valid_0's l2: 0.0947015
[2100]	valid_0's l2: 0.0941887
[2200]	valid_0's l2: 0.0937207
[2300]	valid_0's l2: 0.0932925
[2400]	valid_0's l2: 0.0929
[2500]	valid_0's l2: 0.0925379
[2600]	valid_0's l2: 0.0922065
[2700]	valid_0's l2: 0.0919032
[2800]	valid_0's l2: 0.0916256
[2900]	valid_0's l2: 0.0913684
[3000]	valid_0's l2: 0.0911345
[3100]	valid_0's l2: 0.0909182
[

0.541219083180964

In [216]:
X_train, X_test, y_train, y_test = train_test_split(train[features_b], target.target_b, test_size=0.25, shuffle=False, random_state=2021)

model_target_b = lgb.LGBMRegressor(**params_b, random_state=2021)

model_target_b.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=100, verbose=100)

preds_b = model_target_b.predict(X_test, model_target_b.best_iteration_)

scorer(y_test, preds_b)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.0674594
Early stopping, best iteration is:
[64]	valid_0's l2: 0.0674502
Score as calculated for the leader board (っಠ‿ಠ)っ 0.6814260043800434


0.6814260043800434

In [220]:
prediction = pd.DataFrame()
prediction['target_r'] = model_target_r.predict(test[features_r], num_iteration=model_target_r.best_iteration_)
prediction['target_g'] = model_target_g.predict(test[features_g], num_iteration=model_target_g.best_iteration_)
prediction['target_b'] = model_target_b.predict(test[features_b], num_iteration=model_target_b.best_iteration_)

In [222]:
prediction

,target_r,target_g,target_b
0,0.409018,0.471319,0.361749
1,0.579756,0.613757,0.655615
2,0.422631,0.373635,0.264312
3,0.556723,0.550042,0.607956
4,0.433282,0.464283,0.392723
...,...,...,...
48532,0.671831,0.714529,0.729245
48533,0.727312,0.817538,0.890776
48534,0.686496,0.731974,0.793941
48535,0.727312,0.817538,0.890776


In [223]:
API_KEY = "0GIskIC9Rd1I8sMw6nStutbIbMx7AaZzAyNfuMOVAVWgJtNa1IVQHZAD3WPj" # <- HERE

r = requests.post("https://tournament.datacrunch.com/api/submission",
    files = {
        "file": ("x", prediction.to_csv().encode('ascii'))
    },
    data = {
        "apiKey": API_KEY
    },
)

if r.status_code == 200:
    print("Submission submitted :)")
elif r.status_code == 423:
    print("ERR: Submissions are close")
    print("You can only submit during rounds eg: Friday 7pm GMT+1 to Sunday midnight GMT+1.")
    print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
elif r.status_code == 422:
    print("ERR: API Key is missing or empty")
    print("Did you forget to fill the API_KEY variable?")
elif r.status_code == 404:
    print("ERR: Unknown API Key")
    print("You should check that the provided API key is valid and is the same as the one you've received by email.")
elif r.status_code == 400:
    print("ERR: The file must not be empty")
    print("You have send a empty file.")
elif r.status_code == 401:
    print("ERR: Your email hasn't been verified")
    print("Please verify your email or contact a cruncher.")
elif r.status_code == 429:
    print("ERR: Too many submissions")
else:
    print("ERR: Server returned: " + str(r.status_code))
    print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")

Submission submitted :)
